In [33]:
import os
import pandas as pd

In [34]:
os.getcwd()

'd:\\Studies\\Materials\\Second-cycle\\I year\\II trimester\\ML and StatL\\ML\\ML-project\\src'

In [35]:
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')
mushrooms.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w


In [36]:
mushrooms.dtypes

class                    object
cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-root                object
stem-surface             object
stem-color               object
veil-type                object
veil-color               object
has-ring                 object
ring-type                object
spore-print-color        object
habitat                  object
season                   object
dtype: object

In [37]:
n_rows = mushrooms.shape[0]
n_columns = mushrooms.shape[1]

print(f'There are {n_rows} rows and {n_columns} columns.')

There are 61069 rows and 21 columns.


In [38]:
print("There are no missing values." if not mushrooms.isnull().any().any()\
      else "There are missing values in the dataframe.")

There are missing values in the dataframe.


In [39]:
nan_counts = mushrooms.isna().sum()
nan_percentage = (nan_counts / len(mushrooms)) * 100
nan_summary = pd.DataFrame({
    'Missing Values Count': nan_counts,
    'Percentage of Missing': nan_percentage
})
nan_summary = nan_summary[nan_summary['Missing Values Count'] > 0]
nan_summary

,Missing Values Count,Percentage of Missing
cap-surface,14120,23.121387
gill-attachment,9884,16.184971
gill-spacing,25063,41.040462
stem-root,51538,84.393064
stem-surface,38124,62.427746
veil-type,57892,94.797688
veil-color,53656,87.861272
ring-type,2471,4.046243
spore-print-color,54715,89.595376


In [8]:
edible_count = (mushrooms['class'] == 'e').sum()
poisonous_count = mushrooms.shape[0] - edible_count
print(f"Number of edible mushrooms: {edible_count}\nNumber of poisonous mushrooms: {poisonous_count}")

Number of edible mushrooms: 27181
Number of poisonous mushrooms: 33888


In [9]:
import pandas as pd
from UDFs import DecisionTreeClassifier, train_test_partition, accuracy_metric

# Load the data
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

# Encode the target variable as 0-1
mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

# Drop the missing values
#mushrooms_1 = mushrooms.dropna(axis=1)

# Do not drop the mising values
mushrooms_1 = mushrooms.copy()

# Create X and y
X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

# Partition X and y into training and test sets
X_train, X_test, y_train, y_test = train_test_partition(X, y, test_size=0.3, random_state=42)

# Train the classifier (on the training set)
tree = DecisionTreeClassifier(max_depth=10, criterion="gini", n_features="log2", n_quantiles=10)
tree.fit(X_train, y_train)

# Create predictions on the training set
y_train_predicted = tree.predict(X_train)

# Compute the training error according to the zero-one loss
training_error = 1 - accuracy_metric(y_train, y_train_predicted)
print(f"Training error: {training_error:.2f}")

# Create predictions on the test set
y_test_predicted = tree.predict(X_test)

# Compute the accuracy and test error
accuracy = accuracy_metric(y_test, y_test_predicted)
test_error = 1 - accuracy
print(f"Accuracy (test set): {accuracy:.2f}")
print(f"Test error: {test_error:.2f}")

Training error: 0.15
Accuracy (test set): 0.84
Test error: 0.16


In [10]:
import pandas as pd
import numpy as np
from UDFs import DecisionTreeClassifier, k_fold_partition, accuracy_metric

# Load the data
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

# Encode the target variable as 0-1
mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

# Drop the missing values
#mushrooms_1 = mushrooms.dropna(axis=1)

# Do not drop the mising values
mushrooms_1 = mushrooms.copy()

# Create X and y
X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

## External Cross-Validation ##

# Initialize the classifier
tree = DecisionTreeClassifier(max_depth=10, criterion="gini", n_features="log2", n_quantiles=10)

# Partition X and y into k folds
fold_indices = k_fold_partition(X, random_state=42)

# Initialize a list with test errors
test_errors = []

# Initialize a list with accuracies
accuracies = []

# Perform cross-validation and compute the c-v estimate
for train_indices, test_indices in fold_indices:
    
    X_train, y_train = X[train_indices], y[train_indices]
    X_test, y_test = X[test_indices], y[test_indices]
    
    tree.fit(X_train, y_train)
    
    y_test_predicted = tree.predict(X_test)
    
    test_error = 1 - accuracy_metric(y_test, y_test_predicted)
    test_errors.append(test_error)
    
    accuracy = accuracy_metric(y_test, y_test_predicted)
    accuracies.append(accuracy)

mean_test_error = np.mean(test_errors)
mean_accuracy = np.mean(accuracies)

print("K-Fold Cross-Validation Test Errors:", test_errors)
print("Mean Test Error:", mean_test_error)

print("K-Fold Cross-Validation Accuracies:", accuracies)
print("Mean Accuracy:", mean_accuracy)

K-Fold Cross-Validation Test Errors: [np.float64(0.16809956603619092), np.float64(0.12044542700401217), np.float64(0.17350364365839677), np.float64(0.18619503807418325), np.float64(0.15606321133218704)]
Mean Test Error: 0.16086137722099403
K-Fold Cross-Validation Accuracies: [np.float64(0.8319004339638091), np.float64(0.8795545729959878), np.float64(0.8264963563416032), np.float64(0.8138049619258167), np.float64(0.843936788667813)]
Mean Accuracy: 0.839138622779006


In [ ]:
import pandas as pd
from UDFs import DecisionTreeClassifier, k_fold_cv_estimate

# Load the data
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

# Encode the target variable as 0-1
mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

# Drop the missing values
#mushrooms_1 = mushrooms.dropna(axis=1)

# Do not drop the mising values
mushrooms_1 = mushrooms.copy()

# Create X and y
X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

## External Cross-Validation using a Function ##

# Set the parameters and initialize the classifier
parameters = {
    'max_depth': 10,
    'n_features': "log2",
    'criterion': "gini",
    'n_quantiles': 10
    }

# Perform cross-validation and compute the c-v estimate
mean_test_error = k_fold_cv_estimate(X, y, DecisionTreeClassifier, parameters, random_state=42)
print("Mean Test Error:", mean_test_error)

Mean Test Error: 0.16086137722099403


In [13]:
import pandas as pd
from UDFs import train_test_partition, DecisionTreeClassifier, hyperparameter_tuning

# Load the data
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

# Encode the target variable as 0-1
mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

# Drop the missing values
#mushrooms_1 = mushrooms.dropna(axis=1)

# Do not drop the mising values
mushrooms_1 = mushrooms.copy()

# Create X and y
X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

## Training+Test and Internal Cross-Validation ##

# Partition X and y into training and test sets
X_train, X_test, y_train, y_test = train_test_partition(X, y, test_size=0.3, random_state=42)

# Perform cross-validation to find the best hyperparameter values
parameter_grid = {
    'min_samples_split': [2],
    'max_depth': [5, 10],
    'n_features': [3],
    'criterion': ['scaled_entropy'],
    'min_information_gain': [0.0],
    'n_quantiles': [2, 5],
    'isolate_one': [False]
}

best_parameters, best_mean_test_error = hyperparameter_tuning(X_train, y_train, DecisionTreeClassifier, parameter_grid, random_state=42)
print("Best parameters:", best_parameters)
print("Best mean test error:", best_mean_test_error)

Starting hyperparameter tuning with cross-validation...
Evaluating parameter combination 1/4: {'min_samples_split': 2, 'max_depth': 5, 'n_features': 3, 'criterion': 'scaled_entropy', 'min_information_gain': 0.0, 'n_quantiles': 2, 'isolate_one': False}
Evaluating parameter combination 2/4: {'min_samples_split': 2, 'max_depth': 5, 'n_features': 3, 'criterion': 'scaled_entropy', 'min_information_gain': 0.0, 'n_quantiles': 5, 'isolate_one': False}
Evaluating parameter combination 3/4: {'min_samples_split': 2, 'max_depth': 10, 'n_features': 3, 'criterion': 'scaled_entropy', 'min_information_gain': 0.0, 'n_quantiles': 2, 'isolate_one': False}
Evaluating parameter combination 4/4: {'min_samples_split': 2, 'max_depth': 10, 'n_features': 3, 'criterion': 'scaled_entropy', 'min_information_gain': 0.0, 'n_quantiles': 5, 'isolate_one': False}
Hyperparameter tuning completed.
Best parameters: {'min_samples_split': 2, 'max_depth': 10, 'n_features': 3, 'criterion': 'scaled_entropy', 'min_information_g

In [14]:
from UDFs import _parameter_combinations

parameter_grid = {
    'min_samples_split': [2],
    'max_depth': [5, 10],
    'n_features': [3],
    'criterion': ['scaled_entropy'],
    'min_information_gain': [0.0],
    'n_quantiles': [2, 5],
    'isolate_one': [False]
}

for parameters in _parameter_combinations(parameter_grid):
    print(parameters)

{'min_samples_split': 2, 'max_depth': 5, 'n_features': 3, 'criterion': 'scaled_entropy', 'min_information_gain': 0.0, 'n_quantiles': 2, 'isolate_one': False}
{'min_samples_split': 2, 'max_depth': 5, 'n_features': 3, 'criterion': 'scaled_entropy', 'min_information_gain': 0.0, 'n_quantiles': 5, 'isolate_one': False}
{'min_samples_split': 2, 'max_depth': 10, 'n_features': 3, 'criterion': 'scaled_entropy', 'min_information_gain': 0.0, 'n_quantiles': 2, 'isolate_one': False}
{'min_samples_split': 2, 'max_depth': 10, 'n_features': 3, 'criterion': 'scaled_entropy', 'min_information_gain': 0.0, 'n_quantiles': 5, 'isolate_one': False}


In [15]:
import pandas as pd
from UDFs import DecisionTreeClassifier, k_fold_nested_cv

# Load the data
mushrooms = pd.read_csv('../data/secondary_data.csv', delimiter = ';')

# Encode the target variable as 0-1
mushrooms['class'] = mushrooms['class'].map({'e': 0, 'p': 1})

# Drop the missing values
#mushrooms_1 = mushrooms.dropna(axis=1)

# Do not drop the mising values
mushrooms_1 = mushrooms.copy()

# Create X and y
X = mushrooms_1.drop('class', axis=1).values
y = mushrooms_1['class'].values

## Nested Cross-Validation ##
parameter_grid = {
    'min_samples_split': [50, 100],
    'max_depth': [5, 10],
    'n_features': [5, 10],
    'criterion': ['gini'],
    'min_information_gain': [0.0, 0.1],
    'n_quantiles': [2, 5],
    'isolate_one': [False, True]
}

k_fold_nested_cv(X, y, DecisionTreeClassifier, parameter_grid, k=5, random_state=None)

Starting k-fold nested cross-validation with 5 folds...
Iteration 1/5:
Starting hyperparameter tuning with cross-validation...
Evaluating parameter combination 1/64: {'min_samples_split': 50, 'max_depth': 5, 'n_features': 5, 'criterion': 'gini', 'min_information_gain': 0.0, 'n_quantiles': 2, 'isolate_one': False}
Evaluating parameter combination 2/64: {'min_samples_split': 50, 'max_depth': 5, 'n_features': 5, 'criterion': 'gini', 'min_information_gain': 0.0, 'n_quantiles': 2, 'isolate_one': True}
Evaluating parameter combination 3/64: {'min_samples_split': 50, 'max_depth': 5, 'n_features': 5, 'criterion': 'gini', 'min_information_gain': 0.0, 'n_quantiles': 5, 'isolate_one': False}
Evaluating parameter combination 4/64: {'min_samples_split': 50, 'max_depth': 5, 'n_features': 5, 'criterion': 'gini', 'min_information_gain': 0.0, 'n_quantiles': 5, 'isolate_one': True}
Evaluating parameter combination 5/64: {'min_samples_split': 50, 'max_depth': 5, 'n_features': 5, 'criterion': 'gini', 'min

([{'min_samples_split': 50,
   'max_depth': 10,
   'n_features': 10,
   'criterion': 'gini',
   'min_information_gain': 0.0,
   'n_quantiles': 5,
   'isolate_one': False},
  {'min_samples_split': 50,
   'max_depth': 10,
   'n_features': 10,
   'criterion': 'gini',
   'min_information_gain': 0.0,
   'n_quantiles': 5,
   'isolate_one': False},
  {'min_samples_split': 50,
   'max_depth': 10,
   'n_features': 10,
   'criterion': 'gini',
   'min_information_gain': 0.0,
   'n_quantiles': 5,
   'isolate_one': False},
  {'min_samples_split': 50,
   'max_depth': 10,
   'n_features': 10,
   'criterion': 'gini',
   'min_information_gain': 0.0,
   'n_quantiles': 5,
   'isolate_one': False},
  {'min_samples_split': 50,
   'max_depth': 10,
   'n_features': 10,
   'criterion': 'gini',
   'min_information_gain': 0.0,
   'n_quantiles': 2,
   'isolate_one': False}],
 [np.float64(0.05649717514124297),
  np.float64(0.18586751821829195),
  np.float64(0.08752968148694018),
  np.float64(0.10578891345287811),